In [1]:
import pybullet as p
import pybullet_data

# Connect to PyBullet
p.connect(p.GUI)

# Add PyBullet's default data path
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-10)
planeId = p.loadURDF("plane.urdf")
# Load the Husky robot
husky = p.loadURDF("husky/husky.urdf", basePosition=[0, 0, 0])

# Load the Panda robot arm
panda = p.loadURDF("franka_panda_mod/panda.urdf", basePosition=[0.2, 0, 0.30])  # Adjust basePosition if needed

bat_id = p.loadURDF("Squash.urdf", basePosition=[0, 0, 0.8], baseOrientation = p.getQuaternionFromEuler([0, 0, 0]))  # Replace with the actual path to the URDF


pybullet build time: Feb  4 2024 12:55:26
2025-01-10 22:10:35.857 Python[23719:6496700] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-10 22:10:35.857 Python[23719:6496700] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [2]:
for i in range(p.getNumJoints(husky)):
    print(f"Link {i}: {p.getJointInfo(husky, i)[12].decode('utf-8')}")


Link 0: base_link
Link 1: imu_link
Link 2: front_left_wheel_link
Link 3: front_right_wheel_link
Link 4: rear_left_wheel_link
Link 5: rear_right_wheel_link
Link 6: top_plate_link
Link 7: user_rail_link
Link 8: front_bumper_link
Link 9: rear_bumper_link


In [3]:
# Get the link index of the Husky chassis (replace with actual link index)
husky_chassis_link_index = 0  # Replace with appropriate link index

# Create the fixed constraint
constraint = p.createConstraint(
    parentBodyUniqueId=husky,
    parentLinkIndex=husky_chassis_link_index,
    childBodyUniqueId=panda,
    childLinkIndex=-1,  # -1 means the base link of the child
    jointType=p.JOINT_FIXED,
    jointAxis=[0, 0, 0],
    parentFramePosition=[0.3, 0, 0.25],  # Offset on Husky where the Panda arm will be attached
    childFramePosition=[0, 0, 0]  # Offset on Panda where it attaches to Husky
)


In [4]:
import time
while True:
    p.stepSimulation()
    time.sleep(1/240.0)

KeyboardInterrupt: 

In [22]:
import time

# Set target positions for the joints
target_positions = [-0.066, -0.925, -0.033, -2.016, 0.099, 1.18, 1.818]  # Replace with desired values

# Apply position control
for joint_index, target_position in zip(range(0, 7), target_positions):
    p.setJointMotorControl2(
        bodyUniqueId=panda,
        jointIndex=joint_index,
        controlMode=p.POSITION_CONTROL,
        targetPosition=target_position,
        force=500  # Adjust max torque/force as needed
    )

# Run the simulation loop
while True:
    p.stepSimulation()
    time.sleep(1 / 240)


KeyboardInterrupt: 

In [23]:
for i in range(p.getNumJoints(panda)):
    joint_info = p.getJointInfo(panda, i)
    print(f"Joint {i}: {joint_info[1].decode('utf-8')}")


Joint 0: panda_joint1
Joint 1: panda_joint2
Joint 2: panda_joint3
Joint 3: panda_joint4
Joint 4: panda_joint5
Joint 5: panda_joint6
Joint 6: panda_joint7
Joint 7: panda_joint8


In [8]:
# Replace with the actual link index of the Panda arm's end effector
end_effector_index = 7  # Replace with your actual end effector index

# Position and orientation offsets to align the tennis bat with the end effector
offset_position = [0, 0, 0.2]  # Adjust based on your setup
offset_orientation = p.getQuaternionFromEuler([0, 0, 0])  # Adjust if needed

# Create a fixed constraint
constraint2 = p.createConstraint(
    parentBodyUniqueId=panda,
    parentLinkIndex=end_effector_index,
    childBodyUniqueId=bat_id,
    childLinkIndex=-1,  # Base of the tennis bat
    jointType=p.JOINT_FIXED,
    jointAxis=[0, 0, 0],
    parentFramePosition=offset_position,
    parentFrameOrientation=offset_orientation,
    childFramePosition=[0, 0, 0],
    childFrameOrientation=[0, 0, 0]
)


In [9]:
# p.removeConstraint(constraint2)

In [32]:
# Define wheel indices (replace with your actual indices)
# Replace with actual wheel joint indices

# Set wheel velocities for forward motion
velocity = -5.0  # Adjust as needed
p.setJointMotorControl2(
    bodyUniqueId=husky,
    jointIndex=2,
    controlMode=p.VELOCITY_CONTROL,
    targetVelocity=velocity
)
p.setJointMotorControl2(
    bodyUniqueId=husky,
    jointIndex=3,
    controlMode=p.VELOCITY_CONTROL,
    targetVelocity=velocity
)
p.setJointMotorControl2(
    bodyUniqueId=husky,
    jointIndex=4,
    controlMode=p.VELOCITY_CONTROL,
    targetVelocity=velocity
)
p.setJointMotorControl2(
    bodyUniqueId=husky,
    jointIndex=5,
    controlMode=p.VELOCITY_CONTROL,
    targetVelocity=velocity
)


In [10]:
# panda_joint_indices = [0, 1, 2, 3, 4, 5, 6]  # Adjust based on your URDF

# joint_sliders = []
# for joint_index in panda_joint_indices:
#     joint_sliders.append(p.addUserDebugParameter(f"Joint {joint_index}", -3.14, 3.14, 0))

# while True:
#     for slider, joint_index in zip(joint_sliders, panda_joint_indices):
#         target_position = p.readUserDebugParameter(slider)
#         p.setJointMotorControl2(
#             bodyUniqueId=panda,
#             jointIndex=joint_index,
#             controlMode=p.POSITION_CONTROL,
#             targetPosition=target_position
#         )
#     p.stepSimulation()
#     time.sleep(1/240)


In [33]:
import time
while True:
    p.stepSimulation()
    time.sleep(1/240.0)

KeyboardInterrupt: 